# Testing BCs

Testing on (Bounded, Bounded, Bounded)

In [18]:
using Oceananigans, Oceananigans.Advection, Oceananigans.Fields, Oceananigans.Grids
using Oceananigans.AbstractOperations
using Oceananigans.OutputWriters
using JLD2

In [19]:
grid2D = RegularCartesianGrid(topology = (Bounded, Bounded, Bounded),
                              size=(1, 64, 64), 
                              extent=(2π, 2π, 2π))

RegularCartesianGrid{Float64, Bounded, Bounded, Bounded}
                   domain: x ∈ [0.0, 6.283185307179586], y ∈ [8.410780489584512e-18, 6.283185307179587], z ∈ [-6.283185307179586, 5.517472001167444e-16]
                 topology: (Bounded, Bounded, Bounded)
  resolution (Nx, Ny, Nz): (1, 64, 64)
   halo size (Hx, Hy, Hz): (1, 1, 1)
grid spacing (Δx, Δy, Δz): (6.283185307179586, 0.09817477042468103, 0.09817477042468103)

In [20]:
v_bcs = VVelocityBoundaryConditions(grid2D,
           top = BoundaryCondition(Value, 0.0),
        bottom = BoundaryCondition(Value, 0.0),
         north = BoundaryCondition(NormalFlow, 0.0),
         south = BoundaryCondition(NormalFlow, 0.0)
)

Oceananigans.FieldBoundaryConditions (NamedTuple{(:x, :y, :z)}), with boundary conditions
├── x: CoordinateBoundaryConditions{BoundaryCondition{Flux,Nothing},BoundaryCondition{Flux,Nothing}}
├── y: CoordinateBoundaryConditions{BoundaryCondition{NormalFlow,Float64},BoundaryCondition{NormalFlow,Float64}}
└── z: CoordinateBoundaryConditions{BoundaryCondition{Value,Float64},BoundaryCondition{Value,Float64}}

In [7]:
w_bcs = WVelocityBoundaryConditions(grid2D,
        north = BoundaryCondition(Value, 0.0),
        south = BoundaryCondition(Value, 0.0),
          top = BoundaryCondition(NormalFlow, 0.0),
       bottom = BoundaryCondition(NormalFlow, 0.0)
)

Oceananigans.FieldBoundaryConditions (NamedTuple{(:x, :y, :z)}), with boundary conditions
├── x: CoordinateBoundaryConditions{BoundaryCondition{Flux,Nothing},BoundaryCondition{Flux,Nothing}}
├── y: CoordinateBoundaryConditions{BoundaryCondition{Value,Float64},BoundaryCondition{Value,Float64}}
└── z: CoordinateBoundaryConditions{BoundaryCondition{NormalFlow,Float64},BoundaryCondition{NormalFlow,Float64}}

In [8]:
model2D = IncompressibleModel(timestepper = :RungeKutta3,
                              advection = WENO5(),
                                   grid = grid2D,
                               buoyancy = BuoyancyTracer(),
                                tracers = :b,
                    boundary_conditions = (v=v_bcs, w=w_bcs),
                                closure = IsotropicDiffusivity(ν=2e-4, κ=2e-4))

IncompressibleModel{CPU, Float64}(time = 0 seconds, iteration = 0) 
├── grid: RegularCartesianGrid{Float64, Bounded, Bounded, Bounded}(Nx=1, Ny=64, Nz=64)
├── tracers: (:b,)
├── closure: IsotropicDiffusivity{Float64,NamedTuple{(:b,),Tuple{Float64}}}
├── buoyancy: BuoyancyTracer
└── coriolis: Nothing

In [15]:
u, v, w = model2D.velocities;
println(u)
println(v)
println(w)

Field located at (Face, Cell, Cell)
├── data: OffsetArrays.OffsetArray{Float64,3,Array{Float64,3}}, size: (8, 70, 70)
├── grid: RegularCartesianGrid{Float64, Bounded, Bounded, Bounded}(Nx=1, Ny=64, Nz=64)
└── boundary conditions: x=(west=NormalFlow, east=NormalFlow), y=(south=ZeroFlux, north=ZeroFlux), z=(bottom=ZeroFlux, top=ZeroFlux)
Field located at (Cell, Face, Cell)
├── data: OffsetArrays.OffsetArray{Float64,3,Array{Float64,3}}, size: (7, 71, 70)
├── grid: RegularCartesianGrid{Float64, Bounded, Bounded, Bounded}(Nx=1, Ny=64, Nz=64)
└── boundary conditions: x=(west=ZeroFlux, east=ZeroFlux), y=(south=NormalFlow, north=NormalFlow), z=(bottom=Value, top=Value)
Field located at (Cell, Cell, Face)
├── data: OffsetArrays.OffsetArray{Float64,3,Array{Float64,3}}, size: (7, 70, 71)
├── grid: RegularCartesianGrid{Float64, Bounded, Bounded, Bounded}(Nx=1, Ny=64, Nz=64)
└── boundary conditions: x=(west=ZeroFlux, east=ZeroFlux), y=(south=Value, north=Value), z=(bottom=NormalFlow, top=NormalFlow